In [108]:
import PyPDF2
import regex as re
from nltk import word_tokenize 
from nltk.util import ngrams
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from deviation_dictionary import *
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import json
import os
import glob
from tqdm import tqdm
from nltk.stem import PorterStemmer
import pandas as pd
stemmer = PorterStemmer()
os.chdir('/home/uttkarsh/IP/MPC')


In [109]:
def get_text(file):
    text = ''
    with open(file, 'rb') as pdfFileObj:
        pdfReader = PyPDF2.PdfReader(pdfFileObj)
        num_pages = len(pdfReader.pages)
        for i in range(num_pages):
            pageObj = pdfReader.pages[i]
            text += pageObj.extract_text()
        text = text.replace('\n', '')
    return text

def get_string_from_list(list):
    string = ''
    for i in range(len(list)):
        string += list[i]
        if i != len(list)-1:
            string += ' '
    return string

def filter(text):
    token = word_tokenize(text)
    token = [word.lower() for word in token]
    token = [word for word in token if word.isalpha()]
    token = [word for word in token if (word == 'down' or not word in stop_words )]
    # remoev single character
    token = [word for word in token if len(word) > 1]
    # remove th
    token = [word for word in token if word != 'th']
    for i in range(len(token)):
        if token[i] == 'per' and token[i+1] == 'cent':
            token[i] = 'percent'
            token[i+1] = ''
    token = [word for word in token if word != '']
    return token

def get_unigram(token):
    token = [stemmer.stem(word) for word in token]
    return token

def get_bigram(token):
    token = [stemmer.stem(word) for word in token]
    bigram = ngrams(token, 2)
    return list(bigram)

def get_trigram(token):
    token = [stemmer.stem(word) for word in token]
    trigram = ngrams(token, 3)
    return list(trigram)

In [110]:
# high_dict contains the words, for words with size 1 count them in the bigram list

def count_1_length(unigram_list, term):
    count = 0
    for unigram in unigram_list:
        if unigram == term:
            count+=1
    return count

def count_2_length(unigram_list, term1, term2):
    count = 0
    for i in range(len(unigram_list)):
        if unigram_list[i] == term1:
            if i-7 < 0:
                for k in range(i+7):
                    if unigram_list[k] == term2:
                        count += 1
            elif i+7 > len(unigram_list):
                for k in range(i, len(unigram_list)):
                    if unigram_list[k] == term2:
                        count += 1
            else:
                for k in range(i-7, i+7):
                    if unigram_list[k] == term2:
                        count += 1
    return count

def count_length_1(bigram_list, term):
    count = 0
    for bigram in bigram_list:
        if bigram[0] == term:
            count+=1
    return count

def count_length_2(bigram_list, term1, term2):
    count = 0
    for i in range(len(bigram_list)):
        if term1 in bigram_list[i]:
            if i-7 < 0:
                for k in range(i+7):
                    if term2 in bigram_list[k]:
                        count += 1
            elif i+7 > len(bigram_list):
                for k in range(i, len(bigram_list)):
                    if term2 in bigram_list[k]:
                        count += 1
            else:
                for k in range(i-7, i+7):
                    if term2 in bigram_list[k]:
                        count += 1
    return count

In [111]:
def nearest_date_data(df, date):
    year = int(date.split('-')[1])
    month = int(date.split('-')[0])
    new_df = df[df['Year'] == year]
    # select only month that is either 1 or 2 months ahead
    new_df = df[df['Month'].isin([month, month+1, month+2])]
    # return 1st record
    # if df empty increase the year by 1 and select the first record
    if new_df.empty:
        year += 1
        new_df = df[df['Year'] == year]
    return new_df.iloc[0]

In [112]:
def find_score(df, date, alpha):
    new_df = nearest_date_data(df, date)
    score = alpha*(new_df['Current'] - new_df['3m']) + pow(alpha, 4)*(new_df['Current'] - new_df['1y'])
    return score

In [113]:
# opening each file in each folder
def make_high_low():

    file_names = glob.glob('*.pdf')
    # sort according to year filename using lambda function
    file_names.sort(key = lambda x: x.split('-')[1])
    print(file_names)
    df = pd.read_excel('EHI.xlsx')
    for file_name in file_names:
        file_name = file_name.split('.')[0]
        hd = {}
        ld = {}
        # print(file_name)
        text = get_text(file_name + '.pdf')
        token = filter(text)
        # dump tokens
        with open('tokens', 'w') as fp:
            json.dump(token, fp, indent = 4)
        # N = len(token)
        string_list = get_string_from_list(token)
        uni_list = get_unigram(token)
        bigram_list = get_bigram(token)
        for i in range(len(high_dict)):
            if(len(high_dict[i].split(' ')) == 1):
                term = high_dict[i]
                term = stemmer.stem(term)
                hd[high_dict[i]] = count_1_length(uni_list, high_dict[i])
            elif(len(high_dict[i].split(' ')) == 2):
                term1 = high_dict[i]
                term1 = stemmer.stem(term1.split(' ')[0])
                term2 = high_dict[i]
                term2 = stemmer.stem(term2.split(' ')[1])
                hd[high_dict[i]] = count_2_length(uni_list, term1, term2)
            
        for i in range(len(low_dict)):
            if(len(low_dict[i].split(' ')) == 1):
                term = low_dict[i]
                term = stemmer.stem(term)
                ld[low_dict[i]] = count_1_length(uni_list, low_dict[i]) 
            elif(len(low_dict[i].split(' ')) == 2):
                term1 = low_dict[i]
                term1 = stemmer.stem(term1.split(' ')[0])
                term2 = low_dict[i]
                term2 = stemmer.stem(term2.split(' ')[1])
                ld[low_dict[i]] = count_2_length(uni_list, term1, term2)

        high = sum(hd.values())
        low = sum(ld.values())

        if(high+low == 0):
            score = 0
            continue

        score = (high-low)/(high+low)
        
        new_score = find_score(df, file_name, 0.5)
        print(f"{file_name}, {score}, {new_score}")

        final_dict = {"high": hd, "low": ld,'score': score, 'new_score': new_score}

        file = file_name + '.json'

        with open(file, 'w') as fp:
            json.dump(final_dict, fp, indent = 4)

    return None

In [114]:
make_high_low()

['10-2016.pdf', '12-2016.pdf', '6-2017.pdf', '4-2017.pdf', '12-2017.pdf', '8-2017.pdf', '2-2017.pdf', '10-2017.pdf', '2-2018.pdf', '12-2018.pdf', '6-2018.pdf', '4-2018.pdf', '10-2018.pdf', '8-2018.pdf', '6-2019.pdf', '8-2019.pdf', '2-2019.pdf', '10-2019.pdf', '12-2019.pdf', '4-2019.pdf', '12-2020.pdf', '8-2020.pdf', '3-2020.pdf', '2-2020.pdf', '5-2020.pdf', '10-2020.pdf', '4-2020.pdf', '8-2021.pdf', '6-2021.pdf', '12-2021.pdf', '4-2021.pdf', '2-2021.pdf', '10-2021.pdf', '2-2022.pdf', '9-2022.pdf', '4-2022.pdf', '12-2022.pdf', '6-2022.pdf', '8-2022.pdf', '8-2023.pdf', '4-2023.pdf', '6-2023.pdf', '2-2023.pdf']


10-2016, -0.4444444444444444, 0.18125000000000024
12-2016, 0.16666666666666666, 0.18125000000000024
6-2017, -0.2, -0.30625
4-2017, 0.24324324324324326, -0.30625
12-2017, 0.38461538461538464, 0.18125000000000024
8-2017, -0.2830188679245283, -0.21874999999999944
2-2017, 0.3333333333333333, -0.11249999999999982
10-2017, 0.2727272727272727, 0.18125000000000024
2-2018, 0.6078431372549019, -0.11249999999999982
12-2018, 0.4375, 0.18125000000000024
6-2018, 0.5666666666666667, -0.30625
4-2018, 0.3333333333333333, -0.30625
10-2018, 0.5909090909090909, 0.18125000000000024
8-2018, 0.16981132075471697, -0.21874999999999944
6-2019, 0.0, -0.30625
8-2019, -1.0, -0.21874999999999944
2-2019, -0.3333333333333333, -0.11249999999999982
10-2019, -0.2, 0.18125000000000024
12-2019, -0.14285714285714285, 0.18125000000000024
4-2019, -0.42857142857142855, -0.30625
12-2020, -1.0, 0.18125000000000024
8-2020, -0.25, -0.21874999999999944
3-2020, -0.6, -0.11249999999999982
2-2020, 0.0, -0.11249999999999982
5-2020, -0